In [164]:
# !pip install altair
# !pip install altair_saver --upgrade
# !npm install -g vega-lite vega-cli canvas
# !pip install vl-convert-python --upgrade

In [165]:
# !pip install selenium --upgrade
# !apt-get install chromium-chromedriver -y

In [166]:
# !apt update
# !apt install ttf-mscorefonts-installer -y
# !apt reinstall fontconfig fontconfig-config libfontconfig1 -y

In [167]:
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Black.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Bold.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Book.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Medium.otf -P /usr/local/share/fonts

In [168]:
# !fc-cache -f

In [302]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import urllib.parse

In [303]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [339]:
SAVE = True
LOCAL = False
DARK = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [340]:
%%capture pwd
!pwd

In [341]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
uid=urllib.parse.quote(uid)
if not LOCAL:
    eco_git_home = (
        "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
    )
    vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
    colors = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
    )
    category_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
    )
    hue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
    )
    mhue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
    )
    div_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
    )
    config = json.loads(
        requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
    )
else:
    eco_git_home = '/'.join(pwd.stdout.split("/")[:-2])+'/'
    vega_embed = open(eco_git_home + "guidelines/html/vega-embed.html",'r').read()
    colors = json.load(
        open(eco_git_home + "guidelines/colors/eco-colors.json",'r')
    )
    category_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-category-color.json",'r')
    )
    hue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-single-hue-color.json",'r')
    )
    mhue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-multi-hue-color.json",'r')
    )
    div_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-diverging-color.json",'r')
    )
    config = json.load(
        open(eco_git_home + "guidelines/charts/eco-global-config.json",'r')
    )
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
mo=0.5
height = config["height"]
width = config["width"]
uid, height, width

('how-is-consumer-spending-around-christmas-2023-likely-to-affect-retailers',
 300,
 500)

In [342]:
def save(df, f, LOCAL):
    fc = eco_git_path + f + ".csv"
    df.to_csv("data/" + f + ".csv")
    f += local_suffix
    open("visualisation/" + f + ".html", "w").write(
        vega_embed.replace(
            "JSON_PATH", fc.replace("/data/", "/visualisation/").replace(".csv", ".json")
        )
    )
    if LOCAL:
        fc = df
    
    from IPython.display import display, HTML
    display(HTML(df.head().to_html()))
    
    readme = "## Figure " + f.replace('fig','').split('_')[0] + \
        '  \n\nData: [`csv`](data/' + f + '.csv)' +\
        '  \nGitHub: [' + f + '](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/'+uid +')'+\
        ''+\
        '  \n\n### Light theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local.png) [`svg`](visualisation/' + f + '_local.svg) [`json`](visualisation/' + f + '_local.json) '+ \
        '  \n (**Default**) Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '.png) [`svg`](visualisation/' + f + '.svg) [`json`](visualisation/' + f + '.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding.png) [`svg`](visualisation/' + f + '_local_no_branding.svg) [`json`](visualisation/' + f + '_local_no_branding.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding.png) [`svg`](visualisation/' + f + '_no_branding.svg) [`json`](visualisation/' + f + '_no_branding.json) '+ \
        ''+\
        '  \n\n### Dark theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_dark.png) [`svg`](visualisation/' + f + '_local_dark.svg) [`json`](visualisation/' + f + '_local_dark.json) '+ \
        '  \n Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_dark.png) [`svg`](visualisation/' + f + '_dark.svg) [`json`](visualisation/' + f + '_dark.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding_dark.png) [`svg`](visualisation/' + f + '_local_no_branding_dark.svg) [`json`](visualisation/' + f + '_local_no_branding_dark.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding_dark.png) [`svg`](visualisation/' + f + '_no_branding_dark.svg) [`json`](visualisation/' + f + '_no_branding_dark.json) '+ \
        ''+\
        '  \n\n!["' + f + '"](visualisation/' + f + '.svg "' + f + '")\n\n' +\
        '  \n\n!["' + f + '_dark"](visualisation/' + f + '_dark.svg "' + f + '")\n\n' 
    return readme, f, fc

In [343]:
def area(base,color,opacity=1):
    return base.mark_area(opacity=opacity,
    interpolate="monotone",
    line={'color':color},
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='#ffffff00', offset=0.2),
               alt.GradientStop(color=color, offset=0.8)],
        x1=1, #0.8
        y1=1,
        x2=1,
        y2=0
        )
    )

In [344]:
# service_color='#d6c8da' '#e4bfe2' '#ce4b96' colors['eco-turquiose']
service_color='#b4c8d8'
def dark(f):
    configSource = "visualisation/" + f + ".json"
    config = json.loads(open(configSource, "r").read())
    config['background']=colors['eco-background']
    service_color='#b4c8d8'
    for i in config['layer']:
        if 'encoding' in i:
            for x in ['x','y']:
                if x in i['encoding']:
                    if 'axis' in i['encoding'][x]:
                        for c in ['domainColor','labelColor','tickColor','titleColor','gridColor']:
                            if c in i['encoding'][x]['axis']:
                                i['encoding'][x]['axis'][c]=service_color
        if 'mark' in i:
            if 'color' in i['mark']:
                if i['mark']['color']==colors['eco-gray']:
                    i['mark']['color']=service_color
                elif i['mark']['color']==colors['eco-blue']:
                    i['mark']['color']=colors['eco-yellow']
                elif i['mark']['color']==service_color:
                    i['mark']['color']=colors['eco-green']
                elif 'stops' in i['mark']['color']:
                    for s in i['mark']['color']['stops']:
                        if 'color' in s:
                            if s['color']==colors['eco-gray']:
                                s['color']=service_color
                            elif s['color']==colors['eco-blue']:
                                s['color']=colors['eco-yellow']
                            elif s['color']==service_color:
                                s['color']=colors['eco-green']
            if 'line' in i['mark']:
                if 'color' in i['mark']['line']:
                    if i['mark']['line']['color']==colors['eco-gray']:
                        i['mark']['line']['color']=service_color
                    elif i['mark']['line']['color']==colors['eco-blue']:
                        i['mark']['line']['color']=colors['eco-yellow']
                    elif i['mark']['line']['color']==service_color:
                        i['mark']['line']['color']=colors['eco-green']
    if 'datasets' in config:
        for i in config['datasets']:
            if 'img' in config['datasets'][i][0]:
                if 'eco-icon-dark' in config['datasets'][i][0]['img']:
                    config['datasets'][i][0]['img']=config['datasets'][i][0]['img'].replace('eco-icon-dark','eco-icon-light')
    return alt.Chart.from_dict(config) 

# Fig 1

In [345]:
df = pd.read_csv("raw/Figure_1__CPIH_falls_to_lowest_level_since_November_2021.csv",skiprows=6)
df['time']=pd.to_datetime(df['Unnamed: 0'])
df=df.drop('Unnamed: 0',axis=1)

/tmp/ipykernel_25083/1714898511.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time']=pd.to_datetime(df['Unnamed: 0'])


In [346]:
readme, f, fc = save(df,"fig1_cpi",LOCAL)

,CPIH,CPI,OOH,time
0,2.0,2.2,1.3,2013-10-01
1,1.9,2.1,1.4,2013-11-01
2,1.9,2.0,1.3,2013-12-01
3,1.8,1.9,1.3,2014-01-01
4,1.6,1.7,1.3,2014-02-01


In [347]:
xmin='2013-10-01'
xmax='2023-10-01'
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="%",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
# yaxis=alt.Chart(pd.DataFrame([{'x':'2020-01-01','y':-4},{'x':'2020-01-01','y':14}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(x='x:T',y='y:Q')
# ylabel=alt.Chart(pd.DataFrame([{'x':'2019-01-01','y':-3.2,'t':'↓ decrease'},{'x':'2019-01-01','y':13,'t':'↑ increase'}]))\
#     .mark_text(color=colors["eco-gray"],opacity=mo+0.1,align='left',dx=5).encode(x='x:T',y='y:Q',text='t:N')
# ylabel2=alt.Chart(pd.DataFrame([{'x':'2022-04-01','y':14,'t':'Aged 16-24'}]))\
#     .mark_text(color=colors["eco-gray"],opacity=mo+0.3,align='right',dx=-5,dy=-12,fontSize=12,fontWeight='bold').encode(x='x:T',y='y:Q',text='t:N')

base = alt.Chart(fc).encode(x='time:T')
line1=base.mark_line(color=colors['eco-dot']).encode(y='CPI:Q')
line2=base.mark_line(color=colors['eco-light-blue']).encode(y='CPIH:Q')
line3=base.mark_line(color=colors['eco-turquiose']).encode(y='OOH:Q')
label1=line1.mark_text(text='CPI',fontSize=12,color=colors['eco-dot'],align='left',dx=5).transform_filter("datum.time>toDate('2023-09-01')")
label2=line2.mark_text(text='CPIH',fontSize=12,color=colors['eco-light-blue'],align='left',dx=5,dy=15).transform_filter("datum.time>toDate('2023-09-01')")
label3=line3.mark_text(text='OOH',fontSize=12,color=colors['eco-turquiose'],align='left',dx=5).transform_filter("datum.time>toDate('2023-09-01')")

layer1 = (
    (xaxis+line3+line2+line1+label1+label2+label3).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 14, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:T',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "w").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig1_cpi



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 2

In [348]:
df = pd.read_excel("raw/baserate.xls",sheet_name="HISTORICAL SINCE 1694",skiprows=995).dropna(how='all',axis=1).dropna(how='all',axis=0).ffill()
df.columns=['y','d','m','v']
df['date']=pd.to_datetime(df['y'].astype(int).astype(str)+'-'+df['m'].str.strip()+'-'+df['d'].astype(int).astype(str))

In [349]:
readme, f, fc = save(df,"fig2_boe_rate",LOCAL)

,y,d,m,v,date
0,2006.0,3.0,Aug,4.75,2006-08-03
1,2006.0,9.0,Nov,5.00,2006-11-09
3,2007.0,11.0,Jan,5.25,2007-01-11
4,2007.0,10.0,May,5.50,2007-05-10
5,2007.0,5.0,Jul,5.75,2007-07-05


In [350]:
xmin='2013-10-01'
xmax='2023-10-01'
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
        scale=alt.Scale(domain=[xmin, xmax])
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="%",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
# yaxis=alt.Chart(pd.DataFrame([{'x':'2020-01-01','y':-4},{'x':'2020-01-01','y':14}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(x='x:T',y='y:Q')
# ylabel=alt.Chart(pd.DataFrame([{'x':'2019-01-01','y':-3.2,'t':'↓ decrease'},{'x':'2019-01-01','y':13,'t':'↑ increase'}]))\
#     .mark_text(color=colors["eco-gray"],opacity=mo+0.1,align='left',dx=5).encode(x='x:T',y='y:Q',text='t:N')
# ylabel2=alt.Chart(pd.DataFrame([{'x':'2022-04-01','y':14,'t':'Aged 16-24'}]))\
#     .mark_text(color=colors["eco-gray"],opacity=mo+0.3,align='right',dx=-5,dy=-12,fontSize=12,fontWeight='bold').encode(x='x:T',y='y:Q',text='t:N')

base = alt.Chart(fc).encode(x='date:T')
# line1=base.mark_line(color=colors['eco-dot']).encode(y='CPI:Q')
# line2=base.mark_line(color=colors['eco-light-blue']).encode(y='CPIH:Q')
line3=base.mark_line(color=colors['eco-turquiose'],interpolate='step-after',clip=True).encode(y='v:Q')
# label1=line1.mark_text(text='CPI',fontSize=12,color=colors['eco-dot'],align='left',dx=5).transform_filter("datum.time>toDate('2023-09-01')")
# label2=line2.mark_text(text='CPIH',fontSize=12,color=colors['eco-light-blue'],align='left',dx=5,dy=15).transform_filter("datum.time>toDate('2023-09-01')")
label3=line3.mark_text(text='Bank of England Official Bank Rate',
    fontSize=12,color=colors['eco-turquiose'],align='right',dx=-5).transform_filter("datum.date>toDate('2023-08-01')")

layer1 = (
    (xaxis+line3+label3).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 6, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=-0).encode(x='x:T',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig2_boe_rate



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 3

In [351]:
df = pd.read_csv("raw/Figure_3___Real_earnings_shows_a_positive_annual_growth_rate_for_October_2023.csv",skiprows=6)
df['time']=pd.to_datetime(df['Period'])
df=df.drop('Period',axis=1)

/tmp/ipykernel_25083/3650667825.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time']=pd.to_datetime(df['Period'])


In [352]:
readme, f, fc = save(df,"fig3_pay",LOCAL)

,Total pay (real),Regular pay (real),CPIH,time
0,4.2,2.4,1.2,2001-01-01
1,7.1,2.4,1.2,2001-02-01
2,3.4,3.3,1.3,2001-03-01
3,4.3,3.8,1.5,2001-04-01
4,3.1,2.9,1.9,2001-05-01


In [353]:
xmin='2013-10-01'
xmax='2023-10-01'
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="%",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
# yaxis=alt.Chart(pd.DataFrame([{'x':'2020-01-01','y':-4},{'x':'2020-01-01','y':14}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(x='x:T',y='y:Q')
# ylabel=alt.Chart(pd.DataFrame([{'x':'2019-01-01','y':-3.2,'t':'↓ decrease'},{'x':'2019-01-01','y':13,'t':'↑ increase'}]))\
#     .mark_text(color=colors["eco-gray"],opacity=mo+0.1,align='left',dx=5).encode(x='x:T',y='y:Q',text='t:N')
# ylabel2=alt.Chart(pd.DataFrame([{'x':'2022-04-01','y':14,'t':'Aged 16-24'}]))\
#     .mark_text(color=colors["eco-gray"],opacity=mo+0.3,align='right',dx=-5,dy=-12,fontSize=12,fontWeight='bold').encode(x='x:T',y='y:Q',text='t:N')

base = alt.Chart(fc).encode(x='time:T')
line1=base.mark_line(color=colors['eco-dot']).encode(y='CPIH:Q')
line2=base.mark_line(color=colors['eco-light-blue']).encode(y='Total pay (real):Q')
line3=base.mark_line(color=colors['eco-turquiose']).encode(y='Regular pay (real):Q')
label1=line1.mark_text(text='CPIH',fontSize=12,color=colors['eco-dot'],align='left',dx=5).transform_filter("datum.time>toDate('2023-09-01')")
label2=line2.mark_text(text='Total pay (real)',fontSize=12,color=colors['eco-light-blue'],align='left',dx=5,dy=15).transform_filter("datum.time>toDate('2023-09-01')")
label3=line3.mark_text(text='Regular pay (real)',fontSize=12,color=colors['eco-turquiose'],align='left',dx=5).transform_filter("datum.time>toDate('2023-09-01')")

layer1 = (
    (xaxis+line2+line3+line1+label1+label2+label3).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 14, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:T',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig3_pay



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 4

In [354]:
df = pd.read_csv("raw/asda.csv",header=None)
df['time']=[str(pd.to_datetime('2016-04-01')+pd.to_timedelta(str(i*30.5)+'d'))[:10] for i in df[0]]
df.columns=['d','v','time']
df['v']=df['v'].astype(float)
df=df.drop('d',axis=1)

In [355]:
readme, f, fc = save(df,"fig4_asda",LOCAL)

,v,time
0,15.691964,2015-09-14
1,13.683036,2015-10-17
2,13.080357,2015-11-12
3,13.080357,2015-12-08
4,13.482143,2016-01-10


In [356]:
xmin='2015-10-01'
xmax='2023-10-01'
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
        scale=alt.Scale(domain=[xmin, xmax])
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="£, YoY change",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
# yaxis=alt.Chart(pd.DataFrame([{'x':'2020-01-01','y':-4},{'x':'2020-01-01','y':14}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(x='x:T',y='y:Q')
# ylabel=alt.Chart(pd.DataFrame([{'x':'2019-01-01','y':-3.2,'t':'↓ decrease'},{'x':'2019-01-01','y':13,'t':'↑ increase'}]))\
#     .mark_text(color=colors["eco-gray"],opacity=mo+0.1,align='left',dx=5).encode(x='x:T',y='y:Q',text='t:N')
# ylabel2=alt.Chart(pd.DataFrame([{'x':'2022-04-01','y':14,'t':'Aged 16-24'}]))\
#     .mark_text(color=colors["eco-gray"],opacity=mo+0.3,align='right',dx=-5,dy=-12,fontSize=12,fontWeight='bold').encode(x='x:T',y='y:Q',text='t:N')

base = alt.Chart(fc).encode(x='time:T')
# line1=base.mark_line(color=colors['eco-dot']).encode(y='CPI:Q')
# line2=base.mark_line(color=colors['eco-light-blue']).encode(y='CPIH:Q')
line3=base.mark_line(color=colors['eco-green'],clip=True).encode(y='v:Q')
# label1=line1.mark_text(text='CPI',fontSize=12,color=colors['eco-dot'],align='left',dx=5).transform_filter("datum.time>toDate('2023-09-01')")
# label2=line2.mark_text(text='CPIH',fontSize=12,color=colors['eco-light-blue'],align='left',dx=5,dy=15).transform_filter("datum.time>toDate('2023-09-01')")
label3=line3.mark_text(text='ASDA Income Tracker',
    fontSize=12,color=colors['eco-green'],align='right',dx=-155).transform_filter("datum.time>toDate('2023-09-01')")

layer1 = (
    (xaxis+line3+label3).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 40, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=-0).encode(x='x:T',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig4_asda



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 5

In [357]:
df = pd.DataFrame([{'x':'Lowest Quintile','v':-68.2,'p':0.4},
    {'x':'2nd Quintile','v':5.1,'p':291.4},
    {'x':'3rd Quintile','v':79.5,'p':18.6},
    {'x':'4th Quintile','v':247.9,'p':16.0},
    {'x':'Highest Quintile','v':813.8,'p':9.3}])

In [358]:
readme, f, fc = save(df,"fig5_quint",LOCAL)

,x,v,p
0,Lowest Quintile,-68.2,0.4
1,2nd Quintile,5.1,291.4
2,3rd Quintile,79.5,18.6
3,4th Quintile,247.9,16.0
4,Highest Quintile,813.8,9.3


In [359]:
from urllib.request import urlopen
with urlopen('https://raw.githubusercontent.com/d3/d3-format/master/locale/en-GB.json') as fi:
    de_format = json.load(fi)
alt.renderers.set_embed_options(formatLocale=de_format)

RendererRegistry.enable('default')

In [360]:
base = alt.Chart(fc).encode(x=    alt.X(
        "x:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        )
    ))
bars=base.mark_bar(color=colors['eco-light-blue'],size=55).encode(y= alt.Y(
        "v:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title=["£, average weekly discretionary income by household income", "group for October 2023 (year-on-year growth in brackets)"],
            titleX=0,
            titleY=-22,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        )
    ))
xaxis = alt.Chart(pd.DataFrame([{'x':'Lowest Quintile','y':0},{'x':'Highest Quintile','y':0}]))\
    .mark_line(color=colors["eco-gray"],opacity=mo-0,strokeWidth=1).encode(x=alt.X('x:N',sort=[]),y='y:Q')
labels1=bars.mark_text(fontSize=12,color=colors['eco-light-blue'],dy=-25).encode(text=alt.Text('v:Q',format='$.2f')).transform_filter("datum.x!='Lowest Quintile'")
labels2=labels1.mark_text(fontSize=12,color=colors['eco-light-blue'],dy=-10).encode(text=alt.Text('p2:N')).transform_calculate(p2="'(+'+datum.p+' %)'")
labels1b=bars.mark_text(fontSize=12,color=colors['eco-light-blue'],dy=-45).encode(text=alt.Text('v:Q',format='$.2f')).transform_filter("datum.x=='Lowest Quintile'")
labels2b=labels1b.mark_text(fontSize=12,color=colors['eco-light-blue'],dy=-30).encode(text=alt.Text('p2:N')).transform_calculate(p2="'(+'+datum.p+' %)'")
layer1 = (
    (bars+xaxis+labels1+labels2+labels1b+labels2b).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": "Highest Quintile", "y": 1000, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=75).encode(x=alt.X('x:N',sort=[]),y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig5_quint



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 6

In [361]:
df = pd.read_csv("raw/Figure_4__Food_store_sales_volumes_fall_back_into_decline.csv",skiprows=6)
df['time']=pd.to_datetime(df['Unnamed: 0'])
df=df.drop('Unnamed: 0',axis=1)

/tmp/ipykernel_25083/924556469.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time']=pd.to_datetime(df['Unnamed: 0'])


In [362]:
readme, f, fc = save(df,"fig6_food",LOCAL)

,Value,Volume,time
0,101.0,100.7,2019-10-01
1,100.6,100.3,2019-11-01
2,99.2,99.0,2019-12-01
3,101.8,100.5,2020-01-01
4,100.9,100.0,2020-02-01


In [363]:
xmin='2019-10-01'
xmax='2023-10-01'
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':100},{'x':xmax,'y':100}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
            format='%Y'
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Index, 2019 = 100",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        scale=alt.Scale(domain=[80, 140]),
    )
)
# yaxis=alt.Chart(pd.DataFrame([{'x':'2020-01-01','y':-4},{'x':'2020-01-01','y':14}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(x='x:T',y='y:Q')
# ylabel=alt.Chart(pd.DataFrame([{'x':'2019-01-01','y':-3.2,'t':'↓ decrease'},{'x':'2019-01-01','y':13,'t':'↑ increase'}]))\
#     .mark_text(color=colors["eco-gray"],opacity=mo+0.1,align='left',dx=5).encode(x='x:T',y='y:Q',text='t:N')
# ylabel2=alt.Chart(pd.DataFrame([{'x':'2022-04-01','y':14,'t':'Aged 16-24'}]))\
#     .mark_text(color=colors["eco-gray"],opacity=mo+0.3,align='right',dx=-5,dy=-12,fontSize=12,fontWeight='bold').encode(x='x:T',y='y:Q',text='t:N')

base = alt.Chart(fc).encode(x='time:T')
line1=base.mark_line(color=colors['eco-dot']).encode(y='Value:Q')
# line2=base.mark_line(color=colors['eco-light-blue']).encode(y='Total pay (real):Q')
line3=base.mark_line(color=colors['eco-turquiose']).encode(y='Volume:Q')
label1=line1.mark_text(text='Value',fontSize=12,color=colors['eco-dot'],align='left',dx=5).transform_filter("datum.time>toDate('2023-09-01')")
# label2=line2.mark_text(text='Total pay (real)',fontSize=12,color=colors['eco-light-blue'],align='left',dx=5,dy=15).transform_filter("datum.time>toDate('2023-09-01')")
label3=line3.mark_text(text='Volume',fontSize=12,color=colors['eco-turquiose'],align='left',dx=5).transform_filter("datum.time>toDate('2023-09-01')")

layer1 = (
    (xaxis+line3+line1+label1+label3).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 140, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=0).encode(x='x:T',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig6_food



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 7

In [364]:
df = pd.read_csv("raw/gfk.csv",header=None)
df.columns=['d','v']

In [365]:
readme, f, fc = save(df,"fig6_gfk",LOCAL)

,d,v
0,2013.833869,-12.189795
1,2013.895968,-12.614022
2,2013.931453,-10.818128
3,2013.966938,-8.923247
4,2013.993551,-7.141494


In [366]:
xmin=2013
xmax=2024
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
            format='.0f',
        ),
        scale=alt.Scale(domain=[xmin, xmax])
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
# yaxis=alt.Chart(pd.DataFrame([{'x':'2020-01-01','y':-4},{'x':'2020-01-01','y':14}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(x='x:T',y='y:Q')
# ylabel=alt.Chart(pd.DataFrame([{'x':'2019-01-01','y':-3.2,'t':'↓ decrease'},{'x':'2019-01-01','y':13,'t':'↑ increase'}]))\
#     .mark_text(color=colors["eco-gray"],opacity=mo+0.1,align='left',dx=5).encode(x='x:T',y='y:Q',text='t:N')
# ylabel2=alt.Chart(pd.DataFrame([{'x':'2022-04-01','y':14,'t':'Aged 16-24'}]))\
#     .mark_text(color=colors["eco-gray"],opacity=mo+0.3,align='right',dx=-5,dy=-12,fontSize=12,fontWeight='bold').encode(x='x:T',y='y:Q',text='t:N')

base = alt.Chart(fc).encode(x='d:Q')
# line1=base.mark_line(color=colors['eco-dot']).encode(y='CPI:Q')
# line2=base.mark_line(color=colors['eco-light-blue']).encode(y='CPIH:Q')
line3=base.mark_line(color=colors['eco-turquiose'],clip=True).encode(y='v:Q')
# label1=line1.mark_text(text='CPI',fontSize=12,color=colors['eco-dot'],align='left',dx=5).transform_filter("datum.time>toDate('2023-09-01')")
# label2=line2.mark_text(text='CPIH',fontSize=12,color=colors['eco-light-blue'],align='left',dx=5,dy=15).transform_filter("datum.time>toDate('2023-09-01')")
label3=line3.mark_text(text='ASDA Income Tracker',
    fontSize=12,color=colors['eco-green'],align='right',dx=-155).transform_filter("datum.time>toDate('2023-09-01')")

layer1 = (
    (xaxis+line3+label3).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 10, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=-0).encode(x='x:Q',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig6_gfk



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 8

In [367]:
df=pd.read_excel('raw/shp.xlsx',header=[0,1])
df.columns=[' '.join(i).replace('  ',' ') for i in df.columns]
df=df.set_index(df.columns[0]).stack().reset_index()
df.columns=['cat','gen','v']
df['gen']=df['gen'].str.replace('Unnamed: 1_level_0','').str.replace('Unnamed: 2_level_0','')

In [368]:
readme, f, fc = save(df,"fig8_gifts",LOCAL)

,cat,gen,v
0,Clothing/Apparel,Female,0.593
1,Clothing/Apparel,Male,0.583
2,Clothing/Apparel,Gen Z (born 1997-2012),0.528
3,Clothing/Apparel,Millennials (born 1981-1996),0.627
4,Clothing/Apparel,Gen X (born 1965-1980),0.588


In [369]:
base=alt.Chart(fc).mark_bar().encode(
    y=alt.Y('cat:N',sort=[],
        axis=alt.Axis(
            grid=False,
            title='',
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=0,
            tickOpacity=mo,
            labelOpacity=1,
            labelFontSize=12,
        )),
    x=alt.X('gen:N',sort=[],
        axis=alt.Axis(
            grid=False,
            title="",
            ticks=False,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=0,
            tickOpacity=mo,
            labelOpacity=1,
            labelFontSize=12,
            orient="top",
            labelAngle=-45,
            labelLimit=1000,
        )),
    color=alt.Color('gen:N',legend=None),
    opacity=alt.Opacity('v:Q',legend=None)
)

labels=alt.Chart(fc).mark_text(fontSize=11,color=colors['eco-background']).encode(x='gen:N',y='cat:N',
    text=alt.Text('v:Q',format='.0%'),opacity=alt.Opacity('v:Q',scale=alt.Scale(range=[0.1,0.9])))
layer1 = (
    (base+labels).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": "The Silent Generation (born 1928-1945)", "y": 'Clothing/Apparel', "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='left',baseline='top',yOffset=-10,opacity=mo,xOffset=50).encode(x='x:N',y='y:N',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig8_gifts



alt.LayerChart(...)

alt.LayerChart(...)

# README

In [370]:
from IPython.display import display, Markdown

with open('README.md', 'r') as fh:
    content = fh.read()

display(Markdown(content))

## Figure 1  

Data: [`csv`](data/fig1_cpi.csv)  
GitHub: [fig1_cpi](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-is-consumer-spending-around-christmas-2023-likely-to-affect-retailers)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_cpi_local.png) [`svg`](visualisation/fig1_cpi_local.svg) [`json`](visualisation/fig1_cpi_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig1_cpi.png) [`svg`](visualisation/fig1_cpi.svg) [`json`](visualisation/fig1_cpi.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_cpi_local_no_branding.png) [`svg`](visualisation/fig1_cpi_local_no_branding.svg) [`json`](visualisation/fig1_cpi_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig1_cpi_no_branding.png) [`svg`](visualisation/fig1_cpi_no_branding.svg) [`json`](visualisation/fig1_cpi_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_cpi_local_dark.png) [`svg`](visualisation/fig1_cpi_local_dark.svg) [`json`](visualisation/fig1_cpi_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig1_cpi_dark.png) [`svg`](visualisation/fig1_cpi_dark.svg) [`json`](visualisation/fig1_cpi_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_cpi_local_no_branding_dark.png) [`svg`](visualisation/fig1_cpi_local_no_branding_dark.svg) [`json`](visualisation/fig1_cpi_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig1_cpi_no_branding_dark.png) [`svg`](visualisation/fig1_cpi_no_branding_dark.svg) [`json`](visualisation/fig1_cpi_no_branding_dark.json)   

!["fig1_cpi"](visualisation/fig1_cpi.svg "fig1_cpi")

  

!["fig1_cpi_dark"](visualisation/fig1_cpi_dark.svg "fig1_cpi")

## Figure 2  

Data: [`csv`](data/fig2_boe_rate.csv)  
GitHub: [fig2_boe_rate](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-is-consumer-spending-around-christmas-2023-likely-to-affect-retailers)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_boe_rate_local.png) [`svg`](visualisation/fig2_boe_rate_local.svg) [`json`](visualisation/fig2_boe_rate_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig2_boe_rate.png) [`svg`](visualisation/fig2_boe_rate.svg) [`json`](visualisation/fig2_boe_rate.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_boe_rate_local_no_branding.png) [`svg`](visualisation/fig2_boe_rate_local_no_branding.svg) [`json`](visualisation/fig2_boe_rate_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig2_boe_rate_no_branding.png) [`svg`](visualisation/fig2_boe_rate_no_branding.svg) [`json`](visualisation/fig2_boe_rate_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_boe_rate_local_dark.png) [`svg`](visualisation/fig2_boe_rate_local_dark.svg) [`json`](visualisation/fig2_boe_rate_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig2_boe_rate_dark.png) [`svg`](visualisation/fig2_boe_rate_dark.svg) [`json`](visualisation/fig2_boe_rate_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_boe_rate_local_no_branding_dark.png) [`svg`](visualisation/fig2_boe_rate_local_no_branding_dark.svg) [`json`](visualisation/fig2_boe_rate_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig2_boe_rate_no_branding_dark.png) [`svg`](visualisation/fig2_boe_rate_no_branding_dark.svg) [`json`](visualisation/fig2_boe_rate_no_branding_dark.json)   

!["fig2_boe_rate"](visualisation/fig2_boe_rate.svg "fig2_boe_rate")

  

!["fig2_boe_rate_dark"](visualisation/fig2_boe_rate_dark.svg "fig2_boe_rate")

## Figure 3  

Data: [`csv`](data/fig3_pay.csv)  
GitHub: [fig3_pay](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-is-consumer-spending-around-christmas-2023-likely-to-affect-retailers)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_pay_local.png) [`svg`](visualisation/fig3_pay_local.svg) [`json`](visualisation/fig3_pay_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig3_pay.png) [`svg`](visualisation/fig3_pay.svg) [`json`](visualisation/fig3_pay.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_pay_local_no_branding.png) [`svg`](visualisation/fig3_pay_local_no_branding.svg) [`json`](visualisation/fig3_pay_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig3_pay_no_branding.png) [`svg`](visualisation/fig3_pay_no_branding.svg) [`json`](visualisation/fig3_pay_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_pay_local_dark.png) [`svg`](visualisation/fig3_pay_local_dark.svg) [`json`](visualisation/fig3_pay_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig3_pay_dark.png) [`svg`](visualisation/fig3_pay_dark.svg) [`json`](visualisation/fig3_pay_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_pay_local_no_branding_dark.png) [`svg`](visualisation/fig3_pay_local_no_branding_dark.svg) [`json`](visualisation/fig3_pay_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig3_pay_no_branding_dark.png) [`svg`](visualisation/fig3_pay_no_branding_dark.svg) [`json`](visualisation/fig3_pay_no_branding_dark.json)   

!["fig3_pay"](visualisation/fig3_pay.svg "fig3_pay")

  

!["fig3_pay_dark"](visualisation/fig3_pay_dark.svg "fig3_pay")

## Figure 4  

Data: [`csv`](data/fig4_asda.csv)  
GitHub: [fig4_asda](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-is-consumer-spending-around-christmas-2023-likely-to-affect-retailers)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_asda_local.png) [`svg`](visualisation/fig4_asda_local.svg) [`json`](visualisation/fig4_asda_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig4_asda.png) [`svg`](visualisation/fig4_asda.svg) [`json`](visualisation/fig4_asda.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_asda_local_no_branding.png) [`svg`](visualisation/fig4_asda_local_no_branding.svg) [`json`](visualisation/fig4_asda_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig4_asda_no_branding.png) [`svg`](visualisation/fig4_asda_no_branding.svg) [`json`](visualisation/fig4_asda_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_asda_local_dark.png) [`svg`](visualisation/fig4_asda_local_dark.svg) [`json`](visualisation/fig4_asda_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig4_asda_dark.png) [`svg`](visualisation/fig4_asda_dark.svg) [`json`](visualisation/fig4_asda_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_asda_local_no_branding_dark.png) [`svg`](visualisation/fig4_asda_local_no_branding_dark.svg) [`json`](visualisation/fig4_asda_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig4_asda_no_branding_dark.png) [`svg`](visualisation/fig4_asda_no_branding_dark.svg) [`json`](visualisation/fig4_asda_no_branding_dark.json)   

!["fig4_asda"](visualisation/fig4_asda.svg "fig4_asda")

  

!["fig4_asda_dark"](visualisation/fig4_asda_dark.svg "fig4_asda")

## Figure 5  

Data: [`csv`](data/fig5_quint.csv)  
GitHub: [fig5_quint](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-is-consumer-spending-around-christmas-2023-likely-to-affect-retailers)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_quint_local.png) [`svg`](visualisation/fig5_quint_local.svg) [`json`](visualisation/fig5_quint_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig5_quint.png) [`svg`](visualisation/fig5_quint.svg) [`json`](visualisation/fig5_quint.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_quint_local_no_branding.png) [`svg`](visualisation/fig5_quint_local_no_branding.svg) [`json`](visualisation/fig5_quint_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig5_quint_no_branding.png) [`svg`](visualisation/fig5_quint_no_branding.svg) [`json`](visualisation/fig5_quint_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_quint_local_dark.png) [`svg`](visualisation/fig5_quint_local_dark.svg) [`json`](visualisation/fig5_quint_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig5_quint_dark.png) [`svg`](visualisation/fig5_quint_dark.svg) [`json`](visualisation/fig5_quint_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig5_quint_local_no_branding_dark.png) [`svg`](visualisation/fig5_quint_local_no_branding_dark.svg) [`json`](visualisation/fig5_quint_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig5_quint_no_branding_dark.png) [`svg`](visualisation/fig5_quint_no_branding_dark.svg) [`json`](visualisation/fig5_quint_no_branding_dark.json)   

!["fig5_quint"](visualisation/fig5_quint.svg "fig5_quint")

  

!["fig5_quint_dark"](visualisation/fig5_quint_dark.svg "fig5_quint")

## Figure 6  

Data: [`csv`](data/fig6_food.csv)  
GitHub: [fig6_food](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-is-consumer-spending-around-christmas-2023-likely-to-affect-retailers)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6_food_local.png) [`svg`](visualisation/fig6_food_local.svg) [`json`](visualisation/fig6_food_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig6_food.png) [`svg`](visualisation/fig6_food.svg) [`json`](visualisation/fig6_food.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6_food_local_no_branding.png) [`svg`](visualisation/fig6_food_local_no_branding.svg) [`json`](visualisation/fig6_food_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6_food_no_branding.png) [`svg`](visualisation/fig6_food_no_branding.svg) [`json`](visualisation/fig6_food_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6_food_local_dark.png) [`svg`](visualisation/fig6_food_local_dark.svg) [`json`](visualisation/fig6_food_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig6_food_dark.png) [`svg`](visualisation/fig6_food_dark.svg) [`json`](visualisation/fig6_food_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6_food_local_no_branding_dark.png) [`svg`](visualisation/fig6_food_local_no_branding_dark.svg) [`json`](visualisation/fig6_food_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6_food_no_branding_dark.png) [`svg`](visualisation/fig6_food_no_branding_dark.svg) [`json`](visualisation/fig6_food_no_branding_dark.json)   

!["fig6_food"](visualisation/fig6_food.svg "fig6_food")

  

!["fig6_food_dark"](visualisation/fig6_food_dark.svg "fig6_food")

## Figure 6  

Data: [`csv`](data/fig6_gfk.csv)  
GitHub: [fig6_gfk](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-is-consumer-spending-around-christmas-2023-likely-to-affect-retailers)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6_gfk_local.png) [`svg`](visualisation/fig6_gfk_local.svg) [`json`](visualisation/fig6_gfk_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig6_gfk.png) [`svg`](visualisation/fig6_gfk.svg) [`json`](visualisation/fig6_gfk.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6_gfk_local_no_branding.png) [`svg`](visualisation/fig6_gfk_local_no_branding.svg) [`json`](visualisation/fig6_gfk_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6_gfk_no_branding.png) [`svg`](visualisation/fig6_gfk_no_branding.svg) [`json`](visualisation/fig6_gfk_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6_gfk_local_dark.png) [`svg`](visualisation/fig6_gfk_local_dark.svg) [`json`](visualisation/fig6_gfk_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig6_gfk_dark.png) [`svg`](visualisation/fig6_gfk_dark.svg) [`json`](visualisation/fig6_gfk_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig6_gfk_local_no_branding_dark.png) [`svg`](visualisation/fig6_gfk_local_no_branding_dark.svg) [`json`](visualisation/fig6_gfk_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig6_gfk_no_branding_dark.png) [`svg`](visualisation/fig6_gfk_no_branding_dark.svg) [`json`](visualisation/fig6_gfk_no_branding_dark.json)   

!["fig6_gfk"](visualisation/fig6_gfk.svg "fig6_gfk")

  

!["fig6_gfk_dark"](visualisation/fig6_gfk_dark.svg "fig6_gfk")

## Figure 8  

Data: [`csv`](data/fig8_gifts.csv)  
GitHub: [fig8_gifts](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-is-consumer-spending-around-christmas-2023-likely-to-affect-retailers)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8_gifts_local.png) [`svg`](visualisation/fig8_gifts_local.svg) [`json`](visualisation/fig8_gifts_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig8_gifts.png) [`svg`](visualisation/fig8_gifts.svg) [`json`](visualisation/fig8_gifts.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8_gifts_local_no_branding.png) [`svg`](visualisation/fig8_gifts_local_no_branding.svg) [`json`](visualisation/fig8_gifts_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig8_gifts_no_branding.png) [`svg`](visualisation/fig8_gifts_no_branding.svg) [`json`](visualisation/fig8_gifts_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8_gifts_local_dark.png) [`svg`](visualisation/fig8_gifts_local_dark.svg) [`json`](visualisation/fig8_gifts_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig8_gifts_dark.png) [`svg`](visualisation/fig8_gifts_dark.svg) [`json`](visualisation/fig8_gifts_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig8_gifts_local_no_branding_dark.png) [`svg`](visualisation/fig8_gifts_local_no_branding_dark.svg) [`json`](visualisation/fig8_gifts_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig8_gifts_no_branding_dark.png) [`svg`](visualisation/fig8_gifts_no_branding_dark.svg) [`json`](visualisation/fig8_gifts_no_branding_dark.json)   

!["fig8_gifts"](visualisation/fig8_gifts.svg "fig8_gifts")

  

!["fig8_gifts_dark"](visualisation/fig8_gifts_dark.svg "fig8_gifts")

